In [1]:
import torch
from torch import nn

In [2]:
cfgs = { "A": [64, "M", 128, "M", 256, 256, "M", 512, 512, "M", 512, 512, "M"],
         "B": [64, 64, "M", 128, 128, "M", 256, 256, "M", 512, 512, "M", 512, 512, "M"],
         "D": [64, 64, "M", 128, 128, "M", 256, 256, 256, "M", 512, 512, 512, "M", 512, 512, 512, "M"],
         "E": [64, 64, "M", 128, 128, "M", 256, 256, 256, 256, "M", 512, 512, 512, 512, "M", 512, 512, 512, 512, "M"] }

In [3]:
class VGG(nn.Module):
    def __init__(self, cfg, batch_norm, num_classes = 1000, init_weights = True, drop_p = 0.5):
        super().__init__()

        self.features = self.make_layers(cfg, batch_norm)
        self.avgpool = nn.AdaptiveAvgPool2d((7, 7)) # 7x7 이 되도록 avg pooling 하는 녀석
        self.classifier = nn.Sequential(nn.Linear(512 * 7 * 7, 4096),
                                        nn.ReLU(),
                                        nn.Dropout(p=drop_p),
                                        nn.Linear(4096, 4096),
                                        nn.ReLU(),
                                        nn.Dropout(p=drop_p),
                                        nn.Linear(4096, num_classes))

        if init_weights:
            for m in self.modules():
                if isinstance(m, nn.Conv2d):
                    nn.init.kaiming_normal_(m.weight, mode="fan_out", nonlinearity="relu")
                    if m.bias is not None:
                        nn.init.constant_(m.bias, 0)
                elif isinstance(m, nn.Linear):
                    nn.init.normal_(m.weight, 0, 0.01)
                    nn.init.constant_(m.bias, 0)

    def forward(self, x):
        x = self.features(x)
        x = self.avgpool(x)
        x = torch.flatten(x, 1)
        x = self.classifier(x)
        return x

    def make_layers(self, cfg, batch_norm = False):
        layers = []
        in_channels = 3
        for v in cfg: # cfg = [64, 64, "M", 128, 128, "M", 256, 256, 256, "M", 512, 512, 512, "M", 512, 512, 512, "M"]
            if type(v) == int:
                if batch_norm:
                    layers += [nn.Conv2d(in_channels, v, 3, padding=1), # 어차피 BN에 bias 포함
                               nn.BatchNorm2d(v),
                               nn.ReLU()]
                else:
                    layers += [nn.Conv2d(in_channels, v, 3, padding=1),
                               nn.ReLU()]
                in_channels = v
            else:
                layers += [nn.MaxPool2d(2)]

        return nn.Sequential(*layers)

In [4]:
avgpool = nn.AdaptiveAvgPool2d((4, 4))
print(avgpool(torch.randn(2,3,32,32)).shape)
x = torch.randn(2,3,2,2)
print(avgpool(x)) # 작은 놈이 들어오면 늘려서라도 맞춰준다 # 값을 복제 시켜놓음

torch.Size([2, 3, 4, 4])
tensor([[[[-0.9548, -0.9548, -0.0092, -0.0092],
          [-0.9548, -0.9548, -0.0092, -0.0092],
          [-1.0034, -1.0034, -0.8832, -0.8832],
          [-1.0034, -1.0034, -0.8832, -0.8832]],

         [[-0.9234, -0.9234,  2.1137,  2.1137],
          [-0.9234, -0.9234,  2.1137,  2.1137],
          [ 0.4325,  0.4325,  1.1934,  1.1934],
          [ 0.4325,  0.4325,  1.1934,  1.1934]],

         [[-0.6534, -0.6534,  0.3768,  0.3768],
          [-0.6534, -0.6534,  0.3768,  0.3768],
          [ 1.2509,  1.2509,  0.2538,  0.2538],
          [ 1.2509,  1.2509,  0.2538,  0.2538]]],


        [[[ 2.1370,  2.1370,  1.2564,  1.2564],
          [ 2.1370,  2.1370,  1.2564,  1.2564],
          [-0.4754, -0.4754, -1.0947, -1.0947],
          [-0.4754, -0.4754, -1.0947, -1.0947]],

         [[-2.3182, -2.3182, -0.8298, -0.8298],
          [-2.3182, -2.3182, -0.8298, -0.8298],
          [-0.2303, -0.2303, -1.0157, -1.0157],
          [-0.2303, -0.2303, -1.0157, -1.0157]],

   

In [5]:
model = nn.Sequential(nn.Linear(2,2),
                      nn.ReLU(),
                      nn.Sequential(nn.Linear(2,3),
                                    nn.ReLU(),
                                    nn.Linear(3,3),
                                    nn.ReLU()),
                      nn.Linear(3,10))
[m for m in model.modules()]

[Sequential(
   (0): Linear(in_features=2, out_features=2, bias=True)
   (1): ReLU()
   (2): Sequential(
     (0): Linear(in_features=2, out_features=3, bias=True)
     (1): ReLU()
     (2): Linear(in_features=3, out_features=3, bias=True)
     (3): ReLU()
   )
   (3): Linear(in_features=3, out_features=10, bias=True)
 ),
 Linear(in_features=2, out_features=2, bias=True),
 ReLU(),
 Sequential(
   (0): Linear(in_features=2, out_features=3, bias=True)
   (1): ReLU()
   (2): Linear(in_features=3, out_features=3, bias=True)
   (3): ReLU()
 ),
 Linear(in_features=2, out_features=3, bias=True),
 ReLU(),
 Linear(in_features=3, out_features=3, bias=True),
 ReLU(),
 Linear(in_features=3, out_features=10, bias=True)]

In [6]:
# model1 = nn.Sequential([nn.Linear(1,1),
#                        nn.Linear(1,1)]) # 리스트를 넣으면 안돼요!

model2 = nn.Sequential(nn.Linear(1,1),
                       nn.Linear(1,1))

# print(*[1,2])
# print([1,2])

# model3 = nn.Sequential(nn.Linear(1,1),
#                         nn.Linear(1,1))

In [7]:
model = VGG(cfgs["E"], batch_norm=True)
# print(model)
!pip install torchinfo
from torchinfo import summary
summary(model, input_size=(2,3,224,224), device='cpu')

Layer (type:depth-idx)                   Output Shape              Param #
VGG                                      [2, 1000]                 --
├─Sequential: 1-1                        [2, 512, 7, 7]            --
│    └─Conv2d: 2-1                       [2, 64, 224, 224]         1,792
│    └─BatchNorm2d: 2-2                  [2, 64, 224, 224]         128
│    └─ReLU: 2-3                         [2, 64, 224, 224]         --
│    └─Conv2d: 2-4                       [2, 64, 224, 224]         36,928
│    └─BatchNorm2d: 2-5                  [2, 64, 224, 224]         128
│    └─ReLU: 2-6                         [2, 64, 224, 224]         --
│    └─MaxPool2d: 2-7                    [2, 64, 112, 112]         --
│    └─Conv2d: 2-8                       [2, 128, 112, 112]        73,856
│    └─BatchNorm2d: 2-9                  [2, 128, 112, 112]        256
│    └─ReLU: 2-10                        [2, 128, 112, 112]        --
│    └─Conv2d: 2-11                      [2, 128, 112, 112]        147,

In [8]:
x = torch.randn(2,3,224,224)
print(model(x).shape)

torch.Size([2, 1000])


In [9]:
x = torch.randn(2,3,300,300)
print(model(x).shape)

torch.Size([2, 1000])


In [10]:
x = torch.randn(2,3,32,32)
print(model(x).shape)

torch.Size([2, 1000])
